In [1]:
# TDM 직접구현

docs = ['동물원 코끼리',
       '동물원 원숭이 바나나',
       '엄마 코끼리 아기 코끼리',
       '원숭이 바나나 코끼리 바나나']

In [2]:
doc_ls=[]
for doc in docs:
    doc_ls.append(doc.split(' '))
doc_ls

[['동물원', '코끼리'],
 ['동물원', '원숭이', '바나나'],
 ['엄마', '코끼리', '아기', '코끼리'],
 ['원숭이', '바나나', '코끼리', '바나나']]

In [23]:
from collections import defaultdict

word2id = defaultdict(lambda:len(word2id))

In [24]:
for doc in doc_ls:
    for token in doc:
        word2id[token] # []안의 것은 인덱스라고 이해하면됨
word2id

defaultdict(<function __main__.<lambda>()>,
            {'동물원': 0, '코끼리': 1, '원숭이': 2, '바나나': 3, '엄마': 4, '아기': 5})

In [39]:
import numpy as np

TDM = np.zeros((len(word2id), len(doc_ls)),dtype=int)
print(TDM)
for i,doc in enumerate(doc_ls):
    for token in doc:
        TDM[word2id[token], i] += 1
TDM

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


array([[1, 1, 0, 0],
       [1, 0, 2, 1],
       [0, 1, 0, 1],
       [0, 1, 0, 2],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])

In [28]:
import pandas as pd

doc_names = ['문서' + str(i) for i in range(len(doc_ls))]
print('doc_names',doc_names)
sorted_vocab = sorted((value, key) for key, value in word2id.items())
vocab = [v[1] for v in sorted_vocab]
df_TDM = pd.DataFrame(TDM, columns = doc_names)
df_TDM['단어'] = vocab
df_TDM.set_index('단어')

doc_names ['문서0', '문서1', '문서2', '문서3']


,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
코끼리,1,0,2,1
원숭이,0,1,0,1
바나나,0,1,0,2
엄마,0,0,1,0
아기,0,0,1,0


In [ ]:
# sklearn

docs = ['동물원 코끼리',
       '동물원 원숭이 바나나',
       '엄마 코끼리 아기 코끼리',
       '원숭이 바나나 코끼리 바나나']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
cv = CountVectorizer()

In [30]:
dtm = cv.fit_transform(docs)
dtm.toarray()

array([[1, 0, 0, 0, 0, 1],
       [1, 1, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 2],
       [0, 2, 0, 0, 1, 1]], dtype=int64)

In [31]:
dtm.toarray().T # 행과열 바꾸기

array([[1, 1, 0, 0],
       [0, 1, 0, 2],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 1],
       [1, 0, 2, 1]], dtype=int64)

In [32]:
doc_names = ['문서' + str(i) for i in range(len(doc_ls))]
vocab = cv.get_feature_names()
df_TDM = pd.DataFrame(dtm.toarray().T, columns = doc_names)
df_TDM['단어'] = vocab
df_TDM.set_index('단어')

,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
바나나,0,1,0,2
아기,0,0,1,0
엄마,0,0,1,0
원숭이,0,1,0,1
코끼리,1,0,2,1


In [ ]:
# genism

docs = ['동물원 코끼리',
       '동물원 원숭이 바나나',
       '엄마 코끼리 아기 코끼리',
       '원숭이 바나나 코끼리 바나나']

In [33]:
import gensim
from gensim import corpora

doc_ls = [doc.split() for doc in docs] # 공백으로 토큰화 / 아래 코드를 한 줄로

# doc_ls = []
# for doc in docs:
#    doc_ls.append(doc.split())

print(doc_ls)

[['동물원', '코끼리'], ['동물원', '원숭이', '바나나'], ['엄마', '코끼리', '아기', '코끼리'], ['원숭이', '바나나', '코끼리', '바나나']]


In [35]:
id2word = corpora.Dictionary(doc_ls)
print(id2word)

Dictionary(6 unique tokens: ['동물원', '코끼리', '바나나', '원숭이', '아기']...)


In [36]:
TDM = [id2word.doc2bow(doc) for doc in doc_ls] # 아래 코드를 한줄로

# TDM =[]
# for doc in doc_ls:
#    TDM.append(id2word.docbow(doc))

TDM # 비어있는 것은 값이 없다는 뜻

[[(0, 1), (1, 1)],
 [(0, 1), (2, 1), (3, 1)],
 [(1, 2), (4, 1), (5, 1)],
 [(1, 1), (2, 2), (3, 1)]]

In [38]:
from gensim.matutils import sparse2full

doc_names = ['문서'+ str(i) for i in range(len(doc_ls))]
vocab = [id2word[i] for i in id2word.keys()]
DTM_matrix = [sparse2full(doc, len(vocab)).tolist() for doc in TDM]
print(DTM_matrix)

df_TDM = pd.DataFrame(np.array(DTM_matrix, dtype=int).T,
                     columns=doc_names)

df_TDM['단어'] = vocab
df_TDM.set_index('단어')

[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 1.0, 0.0, 0.0], [0.0, 2.0, 0.0, 0.0, 1.0, 1.0], [0.0, 1.0, 2.0, 1.0, 0.0, 0.0]]


,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
코끼리,1,0,2,1
바나나,0,1,0,2
원숭이,0,1,0,1
아기,0,0,1,0
엄마,0,0,1,0


In [11]:
cv.get_feature_names()

['동물원', '코끼리']

In [13]:
tdm.todense()

matrix([[1, 0],
        [0, 1]], dtype=int64)

In [14]:
tdm = cv.fit_transform(doc_ls[1])
tdm

<3x3 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [15]:
cv.get_feature_names()

['동물원', '바나나', '원숭이']

In [16]:
tdm.todense()

matrix([[1, 0, 0],
        [0, 0, 1],
        [0, 1, 0]], dtype=int64)

In [17]:
tdm = cv.fit_transform(doc_ls[2])
tdm

<4x3 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [18]:
cv.get_feature_names()

['아기', '엄마', '코끼리']

In [19]:
tdm.todense()

matrix([[0, 1, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 0, 1]], dtype=int64)

In [20]:
tdm = cv.fit_transform(doc_ls[3])
tdm

<4x3 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [21]:
cv.get_feature_names()

['바나나', '원숭이', '코끼리']

In [22]:
tdm.todense()

matrix([[0, 1, 0],
        [1, 0, 0],
        [0, 0, 1],
        [1, 0, 0]], dtype=int64)